In [0]:
src = 'abfss://aotst@datalakeeastus2prd.dfs.core.windows.net/aotst_ticket_details/'
df = spark.read.load(src)

In [0]:
df.count()

Out[21]: 2265127963

In [0]:
len(df.columns)

Out[22]: 992

In [0]:
import pandas as pd
dfMap = pd.read_csv("/dbfs/FileStore/tables/aots/MappingTables.csv")
dictCols =  dict(zip(dfMap[dfMap['TABLE_NAME']=='T1913']['FIELDID'],dfMap[dfMap['TABLE_NAME']=='T1913']['FIELDNAME']))
lstColsNew = [] 
lstColsOld = df.columns
for col in lstColsOld:
  #print(col,len(lstColsNew))
  try:
    lstColsNew.append(dictCols[col])
    #print(col,"  replaced with ",dictCols[col])
    #df1 = df1.withColumnRenamed(col,dictCols[col])
  except:
    try:
      lstColsNew.append(dictCols[col.upper()])
      #df1 = df1.withColumnRenamed(col,dictCols[col.upper()])
      #print(col,"  replaced with ",dictCols[col.upper()])
    except:
      #print(col,"  No replacement ")
      lstColsNew.append(col)
      
df = df.toDF(*lstColsNew)

In [0]:
df = df.filter(df['TICKET STATE'] == 'Closed')
df = df.filter(df['VOIP OPTIONS'].like('%IPLD%'))
df = df.filter(~df['VOIP OPTIONS'].like('%IP_CENTREX%'))

In [0]:
df.count()

Out[25]: 391996

In [0]:
df_1 = df.dropDuplicates(subset = ['TICKET  #'])
df_1.count()

Out[26]: 52765

In [0]:
for i in list(" ,;{}()\n\t="):
  NewColumns=(column.replace(i, '_') for column in df.columns)
  df = df.toDF(*NewColumns)

In [0]:
df = df.withColumn('TTTR', df['TICKET_CLEARED_DATE'] - df['TICKET_OPENED'])

In [0]:
df = df.filter(df["TTTR"]>0)
df.count()

Out[29]: 369001

In [0]:
bins = [0,3600*2,3600*4,3600*8,3600*16,3600*24,3600*48,3600*72,3600*24*5,3600*24*7,3600*24*10,float('inf')]
bins_labels = {0.0:"0 to 02 Hrs", 1.0:'02 to 04 Hrs', 2.0:'04 to 08 Hrs', 3.0:'08 to 16 Hrs', 4.0:'16 to 24 Hrs', 5.0:'24 to 48 Hrs', 6.0:'48 to 72 Hrs', 7.0:'72 Hrs to 5 Days', 8.0:'Less than a week,more than 5 Days', 9.0:'More than 01 week,less than 10 Days', 10.0:'More than 10 Days' }

In [0]:
from pyspark.ml.feature import Bucketizer
bucketizer = Bucketizer(splits=bins,inputCol="TTTR", outputCol="TTTR_CAT")
df_buck = bucketizer.setHandleInvalid("keep").transform(df)

In [0]:
df_buck.select("TTTR_CAT").show()

+--------+
TTTR_CAT|
+--------+
 10.0|
 10.0|
 10.0|
 10.0|
 10.0|
 10.0|
 10.0|
 10.0|
 10.0|
 10.0|
 10.0|
 10.0|
 0.0|
 4.0|
 4.0|
 10.0|
 10.0|
 0.0|
 0.0|
 0.0|
+--------+
only showing top 20 rows

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *
udf_foo = udf(lambda x: bins_labels[x], StringType())
df_buck = df_buck.withColumn("TTTR_CAT_1", udf_foo("TTTR_CAT"))

In [0]:
df_buck.select("TTTR_CAT_1").show()

+-----------------+
 TTTR_CAT_1|
+-----------------+
More than 10 Days|
More than 10 Days|
More than 10 Days|
More than 10 Days|
More than 10 Days|
More than 10 Days|
More than 10 Days|
More than 10 Days|
More than 10 Days|
More than 10 Days|
More than 10 Days|
More than 10 Days|
 0 to 02 Hrs|
 16 to 24 Hrs|
 16 to 24 Hrs|
More than 10 Days|
More than 10 Days|
 0 to 02 Hrs|
 0 to 02 Hrs|
 0 to 02 Hrs|
+-----------------+
only showing top 20 rows

In [0]:
df_buck.select("TICKET_OPENED").dtypes

Out[37]: [('TICKET_OPENED', 'string')]

In [0]:
str_col_list = [item[0] for item in df_buck.dtypes if item[1].startswith('string')]
dec_col_list = [item[0] for item in df_buck.dtypes if not item[1].startswith('string')]


In [0]:
len(str_col_list), len(dec_col_list)

Out[40]: (992, 3)

In [0]:
dec_col_list

Out[41]: ['dl_load_ts', 'TTTR', 'TTTR_CAT']

In [0]:
from pyspark.sql.functions import unix_timestamp

In [0]:
df_buck = df_buck.withColumn("TICKET_OPENED",df_buck["TICKET_OPENED"].cast(FloatType()).cast(TimestampType()))

In [0]:
df_buck_1.select("TICKET_OPENED").show()

+-------------------+
 TICKET_OPENED|
+-------------------+
2020-12-22 15:45:04|
2020-12-22 15:45:04|
2020-12-22 15:45:04|
2020-12-22 15:45:04|
2020-12-22 15:45:04|
2020-12-22 15:45:04|
2020-12-22 15:45:04|
2020-12-22 15:45:04|
2020-12-22 15:45:04|
2020-12-22 15:45:04|
2020-12-22 15:45:04|
2020-12-22 15:45:04|
2020-12-23 11:22:40|
2020-12-28 14:32:32|
2020-12-28 14:32:32|
2021-01-11 21:45:36|
2021-01-11 21:45:36|
2021-02-15 18:50:40|
2021-02-15 18:50:40|
2021-02-15 18:50:40|
+-------------------+
only showing top 20 rows

In [0]:
from pyspark.sql.functions import month, year, weekofyear, hour, dayofweek

In [0]:
df_buck = df_buck.withColumn("TICKET_OPENED_Year",year(df_buck["TICKET_OPENED"]) )
df_buck = df_buck.withColumn("TICKET_OPENED_Dayofweek",dayofweek(df_buck["TICKET_OPENED"]) )
df_buck = df_buck.withColumn("TICKET_OPENED_Hour",hour(df_buck["TICKET_OPENED"]) )
df_buck = df_buck.withColumn("TICKET_OPENED_Week",weekofyear(df_buck["TICKET_OPENED"]) )

In [0]:
df_buck.select(["TICKET_OPENED_Year","TICKET_OPENED_Dayofweek", "TICKET_OPENED_Hour", "TICKET_OPENED_Week"]).show()

+------------------+-----------------------+------------------+------------------+
TICKET_OPENED_Year|TICKET_OPENED_Dayofweek|TICKET_OPENED_Hour|TICKET_OPENED_Week|
+------------------+-----------------------+------------------+------------------+
 2020| 3| 15| 52|
 2020| 3| 15| 52|
 2020| 3| 15| 52|
 2020| 3| 15| 52|
 2020| 3| 15| 52|
 2020| 3| 15| 52|
 2020| 3| 15| 52|
 2020| 3| 15| 52|
 2020| 3| 15| 52|
 2020| 3| 15| 52|
 2020| 3| 15| 52|
 2020| 3| 15| 52|
 2020| 4| 11| 52|
 2020| 2| 14| 53|
 2020| 2| 14| 53|
 2021| 2| 21| 2|
 2021| 2| 21| 2|
 2021| 2| 18| 7|
 2021| 2| 18| 7|
 2021| 2| 18| 7|
+------------------+-----------------------+------------------+------------------+
only showing top 20 rows

In [0]:
from pyspark.sql.functions import date_format

df_buck = df_buck.withColumn(
    "is_weekend", 
    date_format("TICKET_OPENED", 'EEE').isin(["Sat", "Sun"]).cast("int")
)

In [0]:
df_buck = df_buck.withColumn("COMPLETION_DEADLINE",df_buck["COMPLETION_DEADLINE"].cast(FloatType()).cast(TimestampType()))
df_buck = df_buck.withColumn("NEXT_CHECK_TIME",df_buck["NEXT_CHECK_TIME"].cast(FloatType()).cast(TimestampType()))

In [0]:
from pyspark.sql.functions import datediff
df_buck = df_buck.withColumn("TENTATIVE_TTTR", datediff(df_buck["COMPLETION_DEADLINE"],df_buck["TICKET_OPENED"]))
df_buck = df_buck.withColumn("NEXT_CHECK", datediff(df_buck["NEXT_CHECK_TIME"], df_buck["TICKET_OPENED"]))


In [0]:
final_features = ["TENTATIVE_TTTR","NEXT_CHECK",'SEVERITY','REPT_SVC_IMPACT_FLAG','REPT_REQUEST_TYPE','REPT_SVC_LINE_CD',
                  'TROUBLE_REPT_CD','TICKET_ROLE','TICKET_SOURCE','TICKET_TYPE',"TICKET_OPENED_Week",'TICKET_OPENED_Hour', 
                  "TICKET_OPENED_Dayofweek",'is_weekend',"TTTR_CAT_1"]

In [0]:
df_final = df_buck.select(*final_features)

In [0]:
from pyspark.sql.functions import col, when, count
df_final.select([count(when(col(c).isNull() , c)).alias(c) for c in df_final.columns]
   ).show()

+--------------+----------+--------+--------------------+-----------------+----------------+---------------+-----------+-------------+-----------+------------------+------------------+-----------------------+----------+
TENTATIVE_TTTR|NEXT_CHECK|SEVERITY|REPT_SVC_IMPACT_FLAG|REPT_REQUEST_TYPE|REPT_SVC_LINE_CD|TROUBLE_REPT_CD|TICKET_ROLE|TICKET_SOURCE|TICKET_TYPE|TICKET_OPENED_Week|TICKET_OPENED_Hour|TICKET_OPENED_Dayofweek|is_weekend|
+--------------+----------+--------+--------------------+-----------------+----------------+---------------+-----------+-------------+-----------+------------------+------------------+-----------------------+----------+
 0| 0| 0| 0| 0| 107| 1| 0| 0| 0| 0| 0| 0| 0|
+--------------+----------+--------+--------------------+-----------------+----------------+---------------+-----------+-------------+-----------+------------------+------------------+-----------------------+----------+

In [0]:
df_final = df_final.na.drop(how = "any")

In [0]:
from pyspark.sql.functions import col, when, count
df_final.select([count(when(col(c).isNull() , c)).alias(c) for c in df_final.columns]
   ).show()

+--------------+----------+--------+--------------------+-----------------+----------------+---------------+-----------+-------------+-----------+------------------+------------------+-----------------------+----------+
TENTATIVE_TTTR|NEXT_CHECK|SEVERITY|REPT_SVC_IMPACT_FLAG|REPT_REQUEST_TYPE|REPT_SVC_LINE_CD|TROUBLE_REPT_CD|TICKET_ROLE|TICKET_SOURCE|TICKET_TYPE|TICKET_OPENED_Week|TICKET_OPENED_Hour|TICKET_OPENED_Dayofweek|is_weekend|
+--------------+----------+--------+--------------------+-----------------+----------------+---------------+-----------+-------------+-----------+------------------+------------------+-----------------------+----------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+--------------+----------+--------+--------------------+-----------------+----------------+---------------+-----------+-------------+-----------+------------------+------------------+-----------------------+----------+

In [0]:
df_final.createGlobalTempView("global_view_ETTR_1")

In [0]:
%sql
CREATE DATABASE Data_Modeling


In [0]:
%sql
CREATE TABLE Data_Modeling.ETTR_1 AS select * from global_temp.global_view_ETTR_1;

num_affected_rows,num_inserted_rows


In [0]:
from pyspark.ml.feature import StringIndexer

In [0]:
str_col_list = [item[0] for item in df_final.dtypes if item[1].startswith('string')]
dec_col_list = [item[0] for item in df_final.dtypes if not item[1].startswith('string')]
str_col_list.remove('TTTR_CAT_1')

In [0]:
cat_training_features = [x + 'Index' for x in str_col_list]
feature_idxer = StringIndexer(inputCols = str_col_list ,
                              outputCols = cat_training_features,
                              handleInvalid="keep")

In [0]:
df_final = feature_idxer.fit(df_final).transform(df_final)

In [0]:
df_final = df_final.select(["TENTATIVE_TTTR","NEXT_CHECK",'SEVERITYIndex','REPT_SVC_IMPACT_FLAGIndex','REPT_REQUEST_TYPEIndex','REPT_SVC_LINE_CDIndex',
                  'TROUBLE_REPT_CDIndex','TICKET_ROLEIndex','TICKET_SOURCEIndex','TICKET_TYPEIndex',"TICKET_OPENED_Week",'TICKET_OPENED_Hour', 
                  "TICKET_OPENED_Dayofweek",'is_weekend',"TTTR_CAT_1"])

In [0]:
display(df_final)

TENTATIVE_TTTR,NEXT_CHECK,SEVERITYIndex,REPT_SVC_IMPACT_FLAGIndex,REPT_REQUEST_TYPEIndex,REPT_SVC_LINE_CDIndex,TROUBLE_REPT_CDIndex,TICKET_ROLEIndex,TICKET_SOURCEIndex,TICKET_TYPEIndex,TICKET_OPENED_Week,TICKET_OPENED_Hour,TICKET_OPENED_Dayofweek,is_weekend,TTTR_CAT_1
55,50,3.0,0.0,0.0,0.0,8.0,0.0,1.0,1.0,52,15,3,0,More than 10 Days
55,50,3.0,0.0,0.0,0.0,8.0,0.0,1.0,1.0,52,15,3,0,More than 10 Days
55,50,3.0,0.0,0.0,0.0,8.0,0.0,1.0,1.0,52,15,3,0,More than 10 Days
55,50,3.0,0.0,0.0,0.0,8.0,0.0,1.0,1.0,52,15,3,0,More than 10 Days
55,50,3.0,0.0,0.0,0.0,8.0,0.0,1.0,1.0,52,15,3,0,More than 10 Days
55,50,3.0,0.0,0.0,0.0,8.0,0.0,1.0,1.0,52,15,3,0,More than 10 Days
55,50,3.0,0.0,0.0,0.0,8.0,0.0,1.0,1.0,52,15,3,0,More than 10 Days
55,50,3.0,0.0,0.0,0.0,8.0,0.0,1.0,1.0,52,15,3,0,More than 10 Days
55,50,3.0,0.0,0.0,0.0,8.0,0.0,1.0,1.0,52,15,3,0,More than 10 Days
55,50,3.0,0.0,0.0,0.0,8.0,0.0,1.0,1.0,52,15,3,0,More than 10 Days


In [0]:
df_final.createGlobalTempView("global_view_ETTR_2")

In [0]:
%sql
CREATE TABLE Data_Modeling.ETTR_2 AS select * from global_temp.global_view_ETTR_2;

num_affected_rows,num_inserted_rows
